In [1]:
from pathlib import Path
import pandas as pd, os
from TELF.pipeline.blocks import DataBundle
from TELF.pipeline import BlockManager
from TELF.pipeline.blocks import HNMFkBlock
from TELF.pipeline.blocks import SBatchBlock

In [2]:
import os
import sys; sys.path.append(os.path.join("..", "..", "scripts"))
from generate_X import gen_data, gen_data_sparse
Xsp = gen_data_sparse(shape=[500, 500], density=0.01)["X"]
X = gen_data(R=4, shape=[200, 200])["X"]

In [ ]:
hnmfk_block = HNMFkBlock()
bundle = DataBundle({ 'X':X,
                     'save_path': Path('./example_results')})

[HNMFk] needs → (X)   provides → (model, save_path)


In [4]:
sbatch_hnmfk = SBatchBlock(
    wrapped_block=HNMFkBlock(),
    venv_type="conda",
    venv_path="dev_artic_fox",
)

[HNMFk] needs → (X)   provides → (model, save_path)
[SBatchWrapper] needs → (X)   provides → (model, save_path)


In [5]:
sbatch_hnmfk.wrapped_block.print_settings()
sbatch_hnmfk.print_settings()

needs: ('X',)
provides: ('model', 'save_path')
conditional_needs: []
init_settings: {'nmfk_params': [{'n_perturbs': 5, 'n_iters': 500, 'epsilon': 0.015, 'n_jobs': -1, 'init': 'nnsvd', 'use_gpu': False, 'save_output': True, 'collect_output': True, 'predict_k_method': 'sill', 'verbose': True, 'nmf_verbose': False, 'transpose': False, 'sill_thresh': 0.8, 'pruned': True, 'nmf_method': 'nmf_fro_mu', 'calculate_error': True, 'predict_k': True, 'use_consensus_stopping': 0, 'calculate_pac': True, 'consensus_mat': True, 'perturb_type': 'uniform', 'perturb_multiprocessing': False, 'perturb_verbose': False, 'simple_plot': True, 'k_search_method': 'bst_pre', 'H_sill_thresh': 0.1, 'clustering_method': 'kmeans', 'device': -1}], 'cluster_on': 'H', 'depth': 1, 'sample_thresh': 5, 'K2': False, 'Ks_deep_min': 1, 'Ks_deep_max': 20, 'Ks_deep_step': 1, 'random_identifiers': False, 'root_node_name': 'Root'}
call_settings: {'Ks': range(1, 21), 'from_checkpoint': True, 'save_checkpoint': True}
tag: 'HNMFk'
ve

In [6]:
pipeline = [
    sbatch_hnmfk
]
mgr = BlockManager(pipeline, bundle, capture_output=False)

Block (tag)                 │ Needs (✓/✗) │ Provides
────────────────────────────────────────────────────
SBatchBlock (SBatchWrapper) │ X           │ ['model', 'save_path']



In [ ]:
final_bundle = mgr()